#### Access Gold Price
Currently access the SPDR price which is then converted to approx gold price with a rough conversion.  
Actual Gold Price ticker was not working:   
gold = yf.download("XAUUSD=X", start="2010-01-01", interval="1mo")

Temporarily use y-finance. Look to find a better alternative extracting actual (accurate_ prices)

In [3]:
import yfinance as yf
import pandas as pd
from pathlib import Path

In [ ]:
# Get the root of your project 
project_root = Path.cwd().parents[1]

# Construct the full path to data/raw
data_dir_to_save = project_root / 'data' / 'raw'
data_dir_to_save.mkdir(parents=True, exist_ok=True)  # creates dirs if needed

SAVE_FILE_PATH = data_dir_to_save / 'gold_prices_raw.csv'


In [5]:
# Define the ticker symbol for gold futures
gold = yf.download("GLD", start="2010-01-01", interval="1mo")

C:\Users\MikeD\AppData\Local\Temp\ipykernel_5888\2870580505.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  gold = yf.download("GLD", start="2010-01-01", interval="1mo")
[*********************100%***********************]  1 of 1 completed


In [6]:
# Download monthly gold prices since 2010-01-01
gold.head()

Price,Close,High,Low,Open,Volume
Ticker,GLD,GLD,GLD,GLD,GLD
Date,,,,,
2010-01-01,105.959999,113.589996,105.180000,109.820000,347218200
2010-02-01,109.430000,110.389999,102.279999,106.639999,368010500
2010-03-01,108.949997,112.180000,106.239998,109.480003,326092400
2010-04-01,115.360001,115.699997,109.919998,109.970001,274413100
2010-05-01,118.879997,122.239998,113.309998,115.910004,504968200


In [7]:
gold.columns

MultiIndex([( 'Close', 'GLD'),
            (  'High', 'GLD'),
            (   'Low', 'GLD'),
            (  'Open', 'GLD'),
            ('Volume', 'GLD')],
           names=['Price', 'Ticker'])

In [8]:
# If multi-index columns, extract Close price
if isinstance(gold.columns, pd.MultiIndex):
    gold = gold['Close']['GLD']
else:
    gold = gold['Close']

In [9]:
# Convert to DataFrame, reset index
gold_df = gold.to_frame(name='Gold_Price_USD').reset_index()


In [10]:
gold_df.head()

,Date,Gold_Price_USD
0,2010-01-01,105.959999
1,2010-02-01,109.430000
2,2010-03-01,108.949997
3,2010-04-01,115.360001
4,2010-05-01,118.879997


In [11]:
# Round close price
gold_df['Gold_Price_USD'] = gold_df['Gold_Price_USD'].round(1)

In [12]:
# gold spdr is ~ 1/10 the actual gld price plus fees
gold_df['Gold_Price_$'] = round(gold_df['Gold_Price_USD'] * 10 * 1.06, 1)

In [13]:
gold_df.sample(20)

,Date,Gold_Price_USD,Gold_Price_$
112,2019-05-01,123.3,1307.0
11,2010-12-01,138.7,1470.2
66,2015-07-01,104.9,1111.9
144,2022-01-01,168.1,1781.9
92,2017-09-01,121.6,1289.0
145,2022-02-01,178.4,1891.0
116,2019-09-01,138.9,1472.3
125,2020-06-01,167.4,1774.4
6,2010-07-01,115.5,1224.3
49,2014-02-01,127.6,1352.6


In [14]:
gold_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186 entries, 0 to 185
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Date            186 non-null    datetime64[ns]
 1   Gold_Price_USD  186 non-null    float64       
 2   Gold_Price_$    186 non-null    float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 4.5 KB


In [15]:
gold_df = gold_df[['Date', 'Gold_Price_$']]

In [16]:
gold_df.to_csv(SAVE_FILE_PATH, index=False)